### Example script for training MPNN-POM model

In [4]:
import sys
import warnings
warnings.filterwarnings('ignore')

In [5]:
parent_dir = "/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024"
sys.path.append(parent_dir)
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)

In [6]:
import deepchem as dc
import os
os.environ['TF_ENABLE_MLIR_OPTIMIZATIONS'] = '1'
from openpom.feat.graph_featurizer import GraphFeaturizer, GraphConvConstants
from openpom.utils.data_utils import get_class_imbalance_ratio
from openpom.models.mpnn_pom import MPNNPOMModel
from datetime import datetime
import torch
import numpy as np
import random
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from utils.util_alignment import set_seeds
import pandas as pd
from constants import *

ModuleNotFoundError: No module named 'fast_transformers'

In [7]:
seed = 2024
set_seeds(2024)

NameError: name 'set_seeds' is not defined

In [4]:
def convert_todf_openpom(embeddings_dataset,cids,subjects=None,y=None):
    embeddings_dataset = pd.DataFrame(embeddings_dataset)
    embeddings_dataset['embeddings'] = embeddings_dataset.loc[:, 0:768].values.tolist()
    embeddings_dataset['CID'] = cids
    if subjects is not None:
        embeddings_dataset['subject'] = subjects
    if y is not None:
        y_dataset = pd.DataFrame(y)
        y_dataset['y'] = y_dataset.loc[:, 0:256].values.tolist()
    
        df = pd.concat([embeddings_dataset, y_dataset], axis=1)
        return df
    else:
        return embeddings_dataset

In [5]:

def embed_mols(input_file):
    # get dataset
    # print(os.getcwd())
    featurizer = GraphFeaturizer()
    smiles_field = 'nonStereoSMILES'
    loader = dc.data.CSVLoader(tasks=[],
                       feature_field=smiles_field,
                       featurizer=featurizer)
    dataset = loader.create_dataset(inputs=[input_file])
    
    embeddings=model.predict_embedding(dataset)
    return embeddings,dataset

In [6]:
def postproce_molembeddings(embeddings,index):
    # molecules_embeddings_penultimate = torch.cat(embeddings)
    df_molecules_embeddings = pd.DataFrame(embeddings, index=index)
    df_molecules_embeddings['Combined'] = df_molecules_embeddings.loc[:, '0':'767'].values.tolist()
    df_molecules_embeddings=df_molecules_embeddings.reset_index()
    return(df_molecules_embeddings)

In [7]:
def prepare_mols_helper(input_file,tasks,mol_type="nonStereoSMILES",index="cid"):
    featurizer = GraphFeaturizer()
    # smiles_field = 'nonStereoSMILES'
    loader = dc.data.CSVLoader(tasks=tasks,
                   feature_field=mol_type,
                   featurizer=featurizer
                          )
    dataset = loader.create_dataset(inputs=[input_file])
    df_mols = pd.read_csv(input_file)
    print(df_mols.columns)

    df_mols_embeddings_original=model.predict_embedding(dataset)
    return df_mols_embeddings_original,dataset

In [8]:
# download curated dataset
# !wget https://raw.githubusercontent.com/ARY2260/openpom/main/openpom/data/curated_datasets/curated_GS_LF_merged_4983.csv

# The curated dataset can also found at `openpom/data/curated_datasets/curated_GS_LF_merged_4983.csv` in the repo.

input_file = '/local_storage/datasets/farzaneh/openpom/data/curated_datasets/curated_GS_LF_merged_4983.csv' # or new downloaded file path

In [9]:
df_gslf = pd.read_csv(input_file)

In [10]:
# get dataset
print(os.getcwd())
featurizer = GraphFeaturizer()
smiles_field = 'nonStereoSMILES'
loader = dc.data.CSVLoader(tasks=gs_lf_tasks,
                   feature_field=smiles_field,
                   featurizer=featurizer)
dataset = loader.create_dataset(inputs=[input_file])
n_tasks = len(dataset.tasks)

In [11]:
# def pom_frame(pom_embeds, y, dir,required_desc ):
#     # pom_embeds = model.predict_embedding(dataset)
#     # y_preds = model.predict(dataset)
#     # required_desc = list(dataset.tasks)
#     type1 = {'floral': '#F3F1F7', 'subs': {'muguet': '#FAD7E6', 'lavender': '#8883BE', 'jasmin': '#BD81B7'}}
#     type2 = {'meaty': '#F5EBE8', 'subs': {'savory': '#FBB360', 'beefy': '#7B382A', 'roasted': '#F7A69E'}}
#     type3 = {'ethereal': '#F2F6EC', 'subs': {'cognac': '#BCE2D2', 'fermented': '#79944F', 'alcoholic': '#C2DA8F'}}
#         
#     # Assuming you have your features in the 'features' array
#     pca = PCA(n_components=2, iterated_power=10)  # You can choose the number of components you want (e.g., 2 for 2D visualization)
#     reduced_features = pca.fit_transform(pom_embeds) # try different variations
# 
#     variance_explained = pca.explained_variance_ratio_
# 
#     # Variance explained by PC1 and PC2
#     variance_pc1 = variance_explained[0]
#     variance_pc2 = variance_explained[1]
# 
#     # if is_preds:
#     #     y = np.where(y_preds>threshold, 1.0, 0.0) # try quartile range (or rank)
#     # else:
#     #     y = dataset.y
# 
#     # Generate grid points to evaluate the KDE on (try kernel convolution)
#     x_grid, y_grid = np.meshgrid(np.linspace(reduced_features[:, 0].min(), reduced_features[:, 0].max(), 500),
#                                  np.linspace(reduced_features[:, 1].min(), reduced_features[:, 1].max(), 500))
#     grid_points = np.vstack([x_grid.ravel(), y_grid.ravel()])
#     def get_kde_values(label):
#         plot_idx = required_desc.index(label)
#         # print(y[:, plot_idx])
#         label_indices = np.where(y[:, plot_idx] == 1)[0]
#         kde_label = gaussian_kde(reduced_features[label_indices].T)
#         kde_values_label = kde_label(grid_points)
#         kde_values_label = kde_values_label.reshape(x_grid.shape)
#         return kde_values_label
#     
#     def plot_contours(type_dictionary, bbox_to_anchor):
#         main_label = list(type_dictionary.keys())[0]
#         plt.contourf(x_grid, y_grid, get_kde_values(main_label), levels=1, colors=['#00000000',type_dictionary[main_label],type_dictionary[main_label]])
#         legend_elements = []
#         for label, color in type_dictionary['subs'].items():
#             plt.contour(x_grid, y_grid, get_kde_values(label), levels=1, colors=color, linewidths=2)
#             legend_elements.append(Patch(facecolor=color, label=label))
#         legend = plt.legend(handles=legend_elements, title=main_label, bbox_to_anchor=bbox_to_anchor)
#         legend.get_frame().set_facecolor(type_dictionary[main_label])
#         plt.gca().add_artist(legend)
# 
#     plt.figure(figsize=(15, 10))
#     plt.title('KDE Density Estimation with Contours in Reduced Space')
#     plt.xlabel(f'Principal Component 1 ({round(variance_pc1*100, ndigits=2)}%)')
#     plt.ylabel(f'Principal Component 2 ({round(variance_pc2*100, ndigits=2)}%)')
#     plot_contours(type_dictionary=type1, bbox_to_anchor = (0.2, 0.8))
#     plot_contours(type_dictionary=type2, bbox_to_anchor = (0.9, 0.4))
#     plot_contours(type_dictionary=type3, bbox_to_anchor = (0.3, 0.1))
#     # plt.colorbar(label='Density')
#     # plt.show()
#     # png_file = os.path.join(dir, 'pom_frame.png')
#     # plt.savefig(png_file)
#     plt.savefig("figs/realign_islands.png")
#     plt.show()
#     plt.close()


In [12]:
# get train valid test splits

randomstratifiedsplitter = dc.splits.RandomStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = randomstratifiedsplitter.train_valid_test_split(dataset, frac_train = 0.8, frac_valid = 0.1, frac_test = 0.1, seed = seed)

In [13]:

train,valid,test=randomstratifiedsplitter.split(dataset, frac_train = 0.8, frac_valid = 0.1, frac_test = 0.1, seed = seed)

In [14]:
df_train_valid_test = pd.DataFrame({'main_idx': train + valid + test,
                   'split': ['train'] * len(train) + ['valid'] * len(valid) + ['test'] * len(test)})


In [15]:
df_train_valid_test.to_csv("gslf-splits-idx-third.csv")

In [16]:
# with open('train_test_split', 'w') as f:
     
#     # using csv.writer method from CSV package
#     write = csv.writer(train)
#     write = csv.writer(test)
#     write = csv.writer(split)

In [17]:
for i in range(len(train)):
    if not np.array_equal(train_dataset.y[i],dataset.y[train[i]]):
        print(i)

for i in range(len(valid)):
    if not np.array_equal(valid_dataset.y[i],dataset.y[valid[i]]):
        print(i)

for i in range(len(test)):
    if not np.array_equal(test_dataset.y[i],dataset.y[test[i]]):
        print(i)


for i in range(len(train)):
    if not np.array_equal(train_dataset.y[i],df_gslf.iloc[train[i]].values[2:].tolist()):
        print(i)

for i in range(len(valid)):
    if not np.array_equal(valid_dataset.y[i],df_gslf.iloc[valid[i]].values[2:].tolist()):
        print(i)

for i in range(len(test)):
    if not np.array_equal(test_dataset.y[i],df_gslf.iloc[test[i]].values[2:].tolist()):
        print(i)
        

In [18]:
train_ratios = get_class_imbalance_ratio(train_dataset)
assert len(train_ratios) == n_tasks

In [19]:
train_dataset.y

In [20]:
# learning_rate = ExponentialDecay(initial_rate=0.001, decay_rate=0.5, decay_steps=32*15, staircase=True)
learning_rate = 0.001

In [21]:
# initialize model
device_name = 'cuda'
model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=128,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 100,
                            edge_hidden_feats = 75,
                            edge_out_feats = 100,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'sum',
                            mode = 'classification',
                            number_atom_features = GraphConvConstants.ATOM_FDIM,
                            number_bond_features = GraphConvConstants.BOND_FDIM,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [392, 392],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.12,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1e-5,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = './examples/experiments',
                            device_name=device_name)

In [22]:
nb_epoch = 150

In [23]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)


In [24]:
model.model_dir

In [25]:
# start_time = datetime.now()
# loss_train =[]
# for epoch in range(1, nb_epoch+1):
#         loss = model.fit(
#               train_dataset,
#               nb_epoch=1,
#               max_checkpoints_to_keep=1,
#               deterministic=False,
#               restore=epoch>1,all_losses=loss_train)
#         train_scores = model.evaluate(train_dataset, [metric])['roc_auc_score']
#         valid_scores = model.evaluate(valid_dataset, [metric])['roc_auc_score']
#         # loss_train.append(loss)
#         print(f"epoch {epoch}/{nb_epoch} ; loss = {loss}; train_scores = {train_scores}; valid_scores = {valid_scores}")
# # model.save_checkpoint()
# end_time = datetime.now()

In [25]:
model.load_from_pretrained(model)

In [26]:
test_scores = model.evaluate(test_dataset, [metric])['roc_auc_score']
# print("time_taken: ", str(end_time-start_time))
print("test_score: ", test_scores)

## Extracting embeddings

## GS-LF

In [32]:
embeddings_dataset=model.predict_embedding(dataset)

In [36]:
cids_gslf= df_gslf.index.values.tolist()

In [44]:
df_embeddings = convert_todf_openpom(embeddings_dataset,cids_gslf,None,dataset.y)
df_embeddings.to_csv('gslf_pom_embeddings_Apr17.csv', index=False)

In [45]:
# df_embeddings17.head(5)

### Sagar

In [32]:

input_file_sagar= '/local_storage/datasets/farzaneh/openpom/data/curated_datasets/curated_sagar_subjects_nonaminus.csv'
df_sagar_temp=pd.read_csv(input_file_sagar)
cids_sagar= df_sagar_temp['cid'].values.tolist()
subjects_sagar= df_sagar_temp['subject'].values.tolist()
sagar_tasks= df_sagar_temp.columns.to_list()[1:16]
df_mols_embeddings_original,sagar_dataset=prepare_mols_helper(input_file_sagar,sagar_tasks)

In [33]:
df_sagar_temp

In [34]:
df_embeddings_sagar = convert_todf_openpom(df_mols_embeddings_original,cids_sagar,subjects_sagar,sagar_dataset.y)
df_embeddings_sagar.to_csv('sagar_pom_embeddings_Apr17.csv', index=False)

### Keller

In [37]:
input_file_keller= '/local_storage/datasets/farzaneh/openpom/data/curated_datasets/curated_keller2016_nona.csv'
df_keller_temp=pd.read_csv(input_file_keller)
keller_tasks= df_keller_temp.columns.to_list()[5:]
cids_keller= df_keller_temp['CID'].values.tolist()
subjects_keller= df_keller_temp['Subject'].values.tolist()
df_mols_embeddings_original_keller,keller_dataset=prepare_mols_helper(input_file_keller,keller_tasks,index="CID")

In [38]:
df_embeddings_keller = convert_todf_openpom(df_mols_embeddings_original_keller,cids_keller,subjects_keller,keller_dataset.y)
df_embeddings_keller.to_csv('keller_pom_embeddings_Apr17.csv', index=False)

### Ravia

In [52]:
input_file_ravia = '/local_storage/datasets/farzaneh/openpom/data/curated_datasets/ravia_molecules.csv'
df_ravia_temp=pd.read_csv(input_file_ravia)
embeddings_ravia,dataset=embed_mols(input_file_ravia)
cids_ravia= df_ravia_temp['CID'].values.tolist()
df_embeddings_ravia = convert_todf_openpom(embeddings_ravia,cids_ravia)
df_embeddings_ravia.to_csv('ravia_pom_embeddings_Apr17.csv', index=False)

### Snitz

In [54]:
input_file_snitz = '/local_storage/datasets/farzaneh/openpom/data/curated_datasets/snitz_molecules.csv'
df_snitz_temp=pd.read_csv(input_file_snitz)
embeddings_snitz,dataset=embed_mols(input_file_snitz)
cids_snitz= df_snitz_temp['CID'].values.tolist()
df_embeddings_snitz = convert_todf_openpom(embeddings_snitz,cids_snitz)
df_embeddings_snitz.to_csv('snitz_pom_embeddings_Apr17.csv', index=False)